In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-

In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.NumberOfWorldwideInternetUsersByRegion
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.NumberOfWorldwideInternetUsersByRegion (
        year INT,
        region VARCHAR(45),
        gender VARCHAR(45),
        usage_rate DOUBLE,
        penetration_percentage INT,
        internet_users DOUBLE,
        frequency_of_access VARCHAR(45),
        individuals VARCHAR(45),
        frequency_of_access_percentage INT
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/NumberOfWorldwideInternetUsersByRegion/'
    """
)

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

DataFrame[]

In [4]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers| 

In [5]:
# Let's look into HDFS

In [6]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideInternetUsersByRegion
    """
).show()

+----+--------------------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|year|              region|gender|usage_rate|penetration_percentage|internet_users|frequency_of_access|individuals|frequency_of_access_percentage|
+----+--------------------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|2009|                Asia|  null|       0.0|                     0|         764.4|               null|       null|                             0|
|2009|              Europe|  null|       0.0|                     0|         425.8|               null|       null|                             0|
|2009|       North America|  null|       0.0|                     0|         259.6|               null|       null|                             0|
|2009|Latin America / C...|  null|       0.0|                     0|         186.9|               null|       null|   

In [7]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.NumberOfWorldwideInternetUsersByRegion
    """
).toPandas()

,col_name,data_type,comment
0,year,int,None
1,region,varchar(45),None
2,gender,varchar(45),None
3,usage_rate,double,None
4,penetration_percentage,int,None
5,internet_users,double,None
6,frequency_of_access,varchar(45),None
7,individuals,varchar(45),None
8,frequency_of_access_percentage,int,None
9,,,


In [8]:
# Let's put the files into HDFS

In [9]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideInternetUsersByRegion
    """
).toPandas()

,year,region,gender,usage_rate,penetration_percentage,internet_users,frequency_of_access,individuals,frequency_of_access_percentage
0,2009,Asia,null,0.0,0,764.40,null,null,0
1,2009,Europe,null,0.0,0,425.80,null,null,0
2,2009,North America,null,0.0,0,259.60,null,null,0
3,2009,Latin America / Caribbean,null,0.0,0,186.90,null,null,0
4,2009,Africa,null,0.0,0,86.20,null,null,0
...,...,...,...,...,...,...,...,...,...
72,2020,North America,null,0.0,0,332.91,null,null,0
73,2020,Latin America / Caribbean,null,0.0,0,467.82,null,null,0
74,2020,Africa,null,0.0,0,566.14,null,null,0
75,2020,Middle East,null,0.0,0,184.86,null,null,0


In [10]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.

spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideInternetUsersByRegion
    """
).show()

+----+--------------------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|year|              region|gender|usage_rate|penetration_percentage|internet_users|frequency_of_access|individuals|frequency_of_access_percentage|
+----+--------------------+------+----------+----------------------+--------------+-------------------+-----------+------------------------------+
|2009|                Asia|  null|       0.0|                     0|         764.4|               null|       null|                             0|
|2009|              Europe|  null|       0.0|                     0|         425.8|               null|       null|                             0|
|2009|       North America|  null|       0.0|                     0|         259.6|               null|       null|                             0|
|2009|Latin America / C...|  null|       0.0|                     0|         186.9|               null|       null|   

In [11]:
spark.stop()